# Problem 96

### Su Doku

In [1]:
import numpy as np

In [2]:
with open('data/p096_sudoku.txt') as f:
    lines = f.read().split()
 
grids = np.empty(shape=(50, 9, 9), dtype='uint8')
for i in range(50):
    for row in range(9):
        for col in range(9):
            grids[i, row, col] = lines[i*11 + row + 2][col]

In [3]:
def get_unions_of_nonzero_values(grid):
    
    idx_zero = np.where(grid == 0)
    row_nonzero_values = [grid[row][grid[row] > 0] for row in range(9)]
    col_nonzero_values = [grid[:, col][grid[:, col] > 0] for col in range(9)]

    block_nonzero_values = [[] for _ in range(3)]
    for i in range(3):
        for j in range(3):
            b = grid[3*i:3*i+3, 3*j:3*j+3]
            block_nonzero_values[i].append(b[b>0])
                    
    for y, x in zip(*idx_zero):
        yield y, x, np.union1d(np.union1d(row_nonzero_values[y], col_nonzero_values[x]), block_nonzero_values[y // 3][x // 3])

In [4]:
def set_straightforward_values(grid):
    
    set_some_values = True
    while set_some_values:
        
        set_some_values = False

        for y, x, union in get_unions_of_nonzero_values(grid):

            if len(union) == 8:
                for v in range(1, 10):
                    if v not in union:
                        grid[y, x] = v
                        set_some_values = True
                        break
                        
            if len(union) == 9:
                return None
            
    return grid

In [5]:
def solve(grid, depth=0):
    
    ok_grid = set_straightforward_values(grid)
    if ok_grid is None:
        return None
    
    if check_valid_solution(grid) == False:
        return None
    
    idx_zero = np.where(grid == 0)
    if len(idx_zero[0]) == 0:
        return grid
    else:

        for h in range(7, 0, -1):
            for y, x, union in get_unions_of_nonzero_values(grid):
                if len(union) == h:
                    
                    # Try to set a value
                    for v in range(1, 10):
                        if v not in union:
                            
                            copy = grid.copy()
                            copy[y, x] = v
                            res = solve(copy, depth+1)
                            if res is not None:
                                return res
                            
                    return None

In [6]:
def check_valid_solution(grid):
    
    for i in range(9):
        
        r = grid[i][grid[i] > 0]
        c = grid[:, i][grid[:, i] > 0]
        
        ru = np.unique(r)
        cu = np.unique(c)
        if len(ru) != len(r):
            return False
        
        if len(cu) != len(c):
            return False
        
    for i in range(3):
        for j in range(3):
            
            g = grid[i*3:i*3+3, j*3:j*3+3].flatten()
            g = g[g > 0]
            
            u = np.unique(g)
            if len(u) != len(g):
                return False

In [7]:
s = 0
for grid in grids:
    g = solve(grid.copy())
    print 'Solved!'
    print g
    
    if check_valid_solution(g) == False:
        print 'NOT A VALID SOLUTION'
        break
    
    print
    print
    
    n = g[0, 0] * 100 + g[0, 1] * 10 + g[0, 2]
    s += n

Solved!
[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]


Solved!
[[2 4 5 9 8 1 3 7 6]
 [1 6 9 2 7 3 5 8 4]
 [8 3 7 5 6 4 2 1 9]
 [9 7 6 1 2 5 4 3 8]
 [5 1 3 4 9 8 6 2 7]
 [4 8 2 7 3 6 9 5 1]
 [3 9 1 6 5 7 8 4 2]
 [7 2 8 3 4 9 1 6 5]
 [6 5 4 8 1 2 7 9 3]]


Solved!
[[4 6 2 8 3 1 9 5 7]
 [7 9 5 4 2 6 1 8 3]
 [3 8 1 7 9 5 4 2 6]
 [1 7 3 9 8 4 2 6 5]
 [6 5 9 3 1 2 7 4 8]
 [2 4 8 5 6 7 3 1 9]
 [9 2 6 1 7 8 5 3 4]
 [8 3 4 2 5 9 6 7 1]
 [5 1 7 6 4 3 8 9 2]]


Solved!
[[1 3 7 2 5 6 8 4 9]
 [9 2 8 3 1 4 5 6 7]
 [4 6 5 8 9 7 3 1 2]
 [6 7 3 5 4 2 9 8 1]
 [8 1 9 6 7 3 2 5 4]
 [5 4 2 1 8 9 7 3 6]
 [2 5 6 7 3 1 4 9 8]
 [3 9 1 4 2 8 6 7 5]
 [7 8 4 9 6 5 1 2 3]]


Solved!
[[5 2 3 8 1 6 7 4 9]
 [7 8 4 5 9 3 1 2 6]
 [6 9 1 4 7 2 8 3 5]
 [2 3 9 1 4 5 6 8 7]
 [4 5 7 2 6 8 9 1 3]
 [1 6 8 9 3 7 2 5 4]
 [3 4 2 7 8 9 5 6 1]
 [9 1 5 6 2 4 3 7 8]
 [8 7 6 3 5 1 4 9 2]]




Solved!
[[4 6 9 1 5 8 3 7 2]
 [7 1 2 4 6 3 8 5 9]
 [5 3 8 2 9 7 6 4 1]
 [9 2 7 6 3 4 5 1 8]
 [3 8 5 7 1 9 4 2 6]
 [1 4 6 5 8 2 7 9 3]
 [6 5 3 9 4 1 2 8 7]
 [2 9 4 8 7 6 1 3 5]
 [8 7 1 3 2 5 9 6 4]]


Solved!
[[3 1 6 5 4 9 2 7 8]
 [9 8 7 3 2 1 6 4 5]
 [4 5 2 6 7 8 9 3 1]
 [5 9 4 2 3 6 8 1 7]
 [2 3 8 4 1 7 5 6 9]
 [6 7 1 9 8 5 3 2 4]
 [8 4 5 1 6 2 7 9 3]
 [1 2 9 7 5 3 4 8 6]
 [7 6 3 8 9 4 1 5 2]]


Solved!
[[5 8 6 1 2 7 9 4 3]
 [7 2 3 4 6 9 8 5 1]
 [4 9 1 8 5 3 2 6 7]
 [1 3 5 9 7 4 6 2 8]
 [2 7 9 6 1 8 5 3 4]
 [6 4 8 5 3 2 1 7 9]
 [9 1 7 2 4 6 3 8 5]
 [3 5 2 7 8 1 4 9 6]
 [8 6 4 3 9 5 7 1 2]]


Solved!
[[9 5 4 2 1 3 6 8 7]
 [6 1 7 5 4 8 9 2 3]
 [8 3 2 7 9 6 5 4 1]
 [7 6 3 8 5 1 2 9 4]
 [1 2 8 9 7 4 3 6 5]
 [5 4 9 3 6 2 1 7 8]
 [2 8 1 6 3 7 4 5 9]
 [4 7 5 1 2 9 8 3 6]
 [3 9 6 4 8 5 7 1 2]]


Solved!
[[1 5 9 7 4 3 8 6 2]
 [2 7 6 5 8 9 4 3 1]
 [3 4 8 6 1 2 7 5 9]
 [6 2 4 9 7 8 3 1 5]
 [9 1 7 2 3 5 6 8 4]
 [5 8 3 1 6 4 2 9 7]
 [4 3 5 8 2 1 9 7 6]
 [8 6 1 4 9 7 5 2 3]
 [7 9 2 3 5 6 1 4 8]]




In [8]:
s

24702